In [29]:
LABELS = {
    'Incomplete Thought',
    'Self Correction',
    'Clarification',
    'Generic Disfluency',
    'Misspeak',
    'Unclear',
    'Overlap'
}

In [30]:
import os

project = "HD_set1_1-7"
review_dir = f"../data/{project}/REVIEW/"
# Get all .json files in REVIEW dir
json_files = [f for f in os.listdir(review_dir) if os.path.splitext(f)[1] == '.json']
test_file = json_files[0]
hoarder_flag = int(test_file[0] == '0')

In [31]:
import json

with open(review_dir + test_file) as f:
    raw_data = json.load(f)
raw_data

{'version': '1',
 'data': {'project': {'id': 'NDE5MzE1ZWM',
   'name': 'HD_set1_1-7',
   'completedAt': '2025-02-26T00:50:59.000Z'},
  'document': {'id': 'f50c25c9-451a-4693-aea5-28081dac0250',
   'name': '001_006.txt'},
  'kinds': ['TOKEN_BASED'],
  'rows': [[{'content': 'Interviewer:\r',
     'tokens': ['Interviewer:'],
     'metadata': []}],
   [{'content': "Yeah. Okay. So the next story, I'm going to read you another story. Here we go. John Doe has several boxes that he keeps in case he needs to store something or decides to move. When the items are delivered to his house he keeps the container the item came in. He is worried that he might get rid of a box that he will need later. Stacks of boxes occupy every room. Before entering his bathroom he must remove several stacks of boxes. His house is extremely cluttered and it's difficult for him to walk in his house. Often, it's difficult for Mr. Doe to find his car keys and wallet, so he'll spend 30 minutes to an hour looking for them

In [32]:
data = raw_data['data']
data.keys()

dict_keys(['project', 'document', 'kinds', 'rows', 'labelerInfo', 'labelSets', 'spanLabels', 'arrowLabels', 'boundingBoxLabels', 'timeLabels', 'comments', 'url'])

In [33]:
data['project'], data['document'], data['kinds']

({'id': 'NDE5MzE1ZWM',
  'name': 'HD_set1_1-7',
  'completedAt': '2025-02-26T00:50:59.000Z'},
 {'id': 'f50c25c9-451a-4693-aea5-28081dac0250', 'name': '001_006.txt'},
 ['TOKEN_BASED'])

In [34]:
INTERVIEWER_NAMES = ["Interviewer", "Rebecca"]
PARTICIPANT_NAMES = ["Participant", "Interviewee"]

In [35]:
row_speakers = [''] * len(data['rows'])
speaker = ""
for i in range(len(data['rows'])):
    row = data['rows'][i]
    for column in row:
        if (column['content'].find(":") != -1):
            slice_with_potential_speaker: str = column['content'].split(":")[0].title()
            speaker_found = False
            for name in INTERVIEWER_NAMES:
                if name in slice_with_potential_speaker:
                    speaker = INTERVIEWER_NAMES[0]
                    break
            # Don't look for participant name if we already found the interviewer
            if not speaker_found: 
                for name in PARTICIPANT_NAMES:
                    if name in slice_with_potential_speaker:
                        speaker = PARTICIPANT_NAMES[0]
                        break
        row_speakers[i] = speaker
row_speakers
# speakers = set([d['speaker'] for row in rows for d in row])
# assert len(set([d['speaker'] for row in rows for d in row])) == 2 # make sure there are only two speakers in each row
# speakers

['Interviewer',
 'Interviewer',
 'Participant',
 'Participant',
 'Interviewer',
 'Interviewer',
 'Participant',
 'Participant',
 'Interviewer',
 'Interviewer',
 'Participant',
 'Participant',
 'Interviewer',
 'Interviewer',
 'Participant',
 'Participant',
 'Interviewer',
 'Interviewer',
 'Participant',
 'Participant',
 'Interviewer',
 'Interviewer',
 'Participant',
 'Participant']

In [36]:
# This is where the meat is at
label_data = data['spanLabels']
label_data

[{'id': '1706871071',
  'labeledBy': 'REVIEWER',
  'labeledByUserId': 7646,
  'acceptedByUserId': None,
  'rejectedByUserId': None,
  'layer': 0,
  'counter': 0,
  'status': 'LABELED',
  'hashCode': 'SPAN:2ioD6do-k6wc54hq7MBd_:0:1:0:159:0:1:0:162:3:0:undefined:undefined',
  'labelName': '2ioD6do-k6wc54hq7MBd_',
  'labelItem': {'id': '2ioD6do-k6wc54hq7MBd_', 'labelName': 'Overlap'},
  'textPosition': {'start': {'row': 1,
    'column': 0,
    'tokenIndex': 159,
    'charIndex': 0},
   'end': {'row': 1, 'column': 0, 'tokenIndex': 162, 'charIndex': 3}},
  'confidenceScore': None},
 {'id': '1690351611',
  'labeledBy': 'REVIEWER',
  'labeledByUserId': 7569,
  'acceptedByUserId': None,
  'rejectedByUserId': None,
  'layer': 0,
  'counter': 0,
  'status': 'LABELED',
  'hashCode': 'SPAN:zynoJJigppWPaLrXIbl7r:0:7:0:22:0:7:0:23:7:0:undefined:undefined',
  'labelName': 'zynoJJigppWPaLrXIbl7r',
  'labelItem': {'id': 'zynoJJigppWPaLrXIbl7r',
   'labelName': 'Incomplete Thought'},
  'textPosition': {

In [37]:
labels_with_speakers = [('', False)] * len(label_data)
for i in range(len(label_data)):
    label = label_data[i]

    row_index = label['textPosition']['start']['row']
    speaker = row_speakers[row_index]
    labels_with_speakers[i] = (label['labelItem']['labelName'], speaker)
labels_with_speakers

[('Overlap', 'Interviewer'),
 ('Incomplete Thought', 'Participant'),
 ('Generic Disfluency', 'Participant'),
 ('Overlap', 'Participant'),
 ('Incomplete Thought', 'Participant')]

In [38]:
from collections import Counter
from itertools import product

cntDict = Counter(labels_with_speakers)
for label, speaker in set(product(LABELS, ['Interviewer', 'Participant'])).difference(cntDict.keys()):
    cntDict[(label, speaker)] = 0
cntDict

Counter({('Incomplete Thought', 'Participant'): 2,
         ('Overlap', 'Interviewer'): 1,
         ('Generic Disfluency', 'Participant'): 1,
         ('Overlap', 'Participant'): 1,
         ('Clarification', 'Interviewer'): 0,
         ('Clarification', 'Participant'): 0,
         ('Incomplete Thought', 'Interviewer'): 0,
         ('Misspeak', 'Interviewer'): 0,
         ('Misspeak', 'Participant'): 0,
         ('Generic Disfluency', 'Interviewer'): 0,
         ('Self Correction', 'Interviewer'): 0,
         ('Unclear', 'Interviewer'): 0,
         ('Self Correction', 'Participant'): 0,
         ('Unclear', 'Participant'): 0})

In [39]:
display_dict = {}
for label in LABELS:
    display_dict[label+'–Interviewer'] = cntDict[(label, 'Interviewer')]
    display_dict[label+'–Participant'] = cntDict[(label, 'Participant')]
    display_dict[label+'–Total'] = cntDict[(label, 'Interviewer')] + cntDict[(label, 'Participant')]
display_dict['Total'] = sum(cntDict.values())
display_dict

{'Misspeak–Interviewer': 0,
 'Misspeak–Participant': 0,
 'Misspeak–Total': 0,
 'Unclear–Interviewer': 0,
 'Unclear–Participant': 0,
 'Unclear–Total': 0,
 'Self Correction–Interviewer': 0,
 'Self Correction–Participant': 0,
 'Self Correction–Total': 0,
 'Overlap–Interviewer': 1,
 'Overlap–Participant': 1,
 'Overlap–Total': 2,
 'Clarification–Interviewer': 0,
 'Clarification–Participant': 0,
 'Clarification–Total': 0,
 'Generic Disfluency–Interviewer': 0,
 'Generic Disfluency–Participant': 1,
 'Generic Disfluency–Total': 1,
 'Incomplete Thought–Interviewer': 0,
 'Incomplete Thought–Participant': 2,
 'Incomplete Thought–Total': 2,
 'Total': 5}

In [40]:
assert not [row for row in data['rows'] if len(row) != 1] # make sure each row is a singleton list
doc_tokens = [row[0]['tokens'] for row in data['rows']]
flat_doc_tokens = [token for row in doc_tokens for token in row]
len(set(flat_doc_tokens)) / len(flat_doc_tokens) # TTR

0.5051124744376279

In [41]:
words = 0
for sent in doc_tokens:
    words += len(sent)
words / len(doc_tokens) # average sentence length

20.375

In [42]:
row = {
    'filename' : data['document']['name'], ''
    'isHoarder' : hoarder_flag, 
    **display_dict
}
row

{'filename': '001_006.txt',
 'isHoarder': 1,
 'Misspeak–Interviewer': 0,
 'Misspeak–Participant': 0,
 'Misspeak–Total': 0,
 'Unclear–Interviewer': 0,
 'Unclear–Participant': 0,
 'Unclear–Total': 0,
 'Self Correction–Interviewer': 0,
 'Self Correction–Participant': 0,
 'Self Correction–Total': 0,
 'Overlap–Interviewer': 1,
 'Overlap–Participant': 1,
 'Overlap–Total': 2,
 'Clarification–Interviewer': 0,
 'Clarification–Participant': 0,
 'Clarification–Total': 0,
 'Generic Disfluency–Interviewer': 0,
 'Generic Disfluency–Participant': 1,
 'Generic Disfluency–Total': 1,
 'Incomplete Thought–Interviewer': 0,
 'Incomplete Thought–Participant': 2,
 'Incomplete Thought–Total': 2,
 'Total': 5}

In [ ]:
import pandas as pd

pd.DataFrame([row])

,filename,isHoarder,Incomplete Thought - Interviewer,Incomplete Thought - Participant,Incomplete Thought - Total,Clarification - Interviewer,Clarification - Participant,Clarification - Total,Generic Disfluency - Interviewer,Generic Disfluency - Participant,...,Unclear - Interviewer,Unclear - Participant,Unclear - Total,Misspeak - Interviewer,Misspeak - Participant,Misspeak - Total,Self Correction - Interviewer,Self Correction - Participant,Self Correction - Total,Total
0,2013_211.txt,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
